In [1]:
import PIL
import torch
import glob as gb
import numpy as np
from PIL import Image
import os
import random
import torch.nn as nn
import pandas as pd 
import torchvision.transforms as transforms

model_path = '/home/angelo/Desktop/Github/Feature-Extractors/'
os.sys.path.append(model_path)

from numpy import dot
from numpy.linalg import norm
import matplotlib.pyplot as plt

random.seed(1)
np.random.seed(1)

In [2]:
# Initialization of models

def initialize_senet50_2048():
    import senet50_ft_pytorch.senet50_ft_dims_2048 as model
    network = model.senet50_ft(weights_path=model_path + 'senet50_ft_pytorch/senet50_ft_dims_2048.pth')
    network.eval()
    return network

def initialize_senet50_256():
    import senet50_256_pytorch.senet50_256 as model
    network = model.senet50_256(weights_path=model_path + 'senet50_256_pytorch/senet50_256.pth')
    network.eval()
    return network

def initialize_senet50_128():
    import senet50_128_pytorch.senet50_128 as model
    network = model.senet50_128(weights_path=model_path + 'senet50_128_pytorch/senet50_128.pth')
    network.eval()
    return network

def initialize_resnet50_128():
    import resnet50_128_pytorch.resnet50_128 as model
    network = model.resnet50_128(weights_path=model_path + 'resnet50_128_pytorch/resnet50_128.pth')
    network.eval()
    return network

In [15]:
def load_image(path='', resize=True, SR=False):
    '''
    VGGFace2 Data has mean = (131.0912, 103.8827, 91.4953)
    and std = (1,1,1) with no scaling
    '''
    mean = (131.0912, 103.8827, 91.4953)
    img = Image.open(path)
    if resize:
        if SR:
            img = img.resize((224,224), resample=PIL.Image.BICUBIC) # SR approach goes here
        else:
            img = img.resize((224,224), resample=PIL.Image.BICUBIC)
    
    img = np.array(img) - mean # Normalizing data before inference according to VGG parameters
    return img

def cos_sim(a,b):
    return 1 - dot(a, b)/(norm(a)*norm(b))

def euc_dis(a,b):
    return norm(a-b)

def l2_error(a,b):
    return sum((a-b)**2) 

def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

def return_batch_embedding(model, facepaths, embedding_size=128):
    batch_size = 10 # As original implementation suggests
    num_faces = len(facepaths)
    face_feats = np.empty((num_faces, embedding_size))
    imgchunks = list(chunks(facepaths, batch_size))

    for c, imgs in enumerate(imgchunks):
        im_array = np.array([load_image(i) for i in imgs])
        f = model(torch.Tensor(im_array.transpose(0, 3, 1, 2)))[1].detach().cpu().numpy()[:, :, 0, 0]
        start = c * batch_size
        end = min((c + 1) * batch_size, num_faces)
        # This is different from the Keras model where the normalization has been done inside the model.
        face_feats[start:end] = f / np.sqrt(np.sum(f ** 2, -1, keepdims=True))
    
    return face_feats

def return_embedding(model, img_path):
    im_array = load_image(img_path).reshape((1,3,224,224))
    f = model(torch.Tensor(im_array))[1].detach().cpu().numpy()[:, :, 0, 0]
    face_feats = f / np.sqrt(np.sum(f ** 2, -1, keepdims=True)) # Normalizing embedding
    return face_feats.reshape(-1)

In [22]:
# Testing consistency of embedding

#senet50 = initialize_senet50_2048()
senet50 = initialize_resnet50_128()

low_res_img_path = '/media/angelo/DATEN/Datasets/CelebA/LR_56/test/018520.jpg'
original_img_path = '/media/angelo/DATEN/Datasets/CelebA/HR/test/018510.jpg'

lr_emb = return_embedding(senet50, low_res_img_path)
hr_emb = return_embedding(senet50, original_img_path)

print('Identity Loss')
print(cos_sim(lr_emb, hr_emb))

Identity Loss
0.1506003737449646


In [23]:
# Experimentation

img_path = '/media/angelo/DATEN/Datasets/ICB-RW/Margin1.3_112/'

gallery_path = gb.glob(img_path + 'gallery/*.jpg')
gallery_names = [str(int(os.path.splitext(name)[0][:-2])) for name in os.listdir(img_path + 'gallery/')]

face_embeddings = return_batch_embedding(senet50, gallery_path)

In [24]:
probe_path = img_path + 'probe/'
student_pool_list = gallery_names

real_embedding = return_embedding(senet50, img_path + 'gallery/00' + student_pool_list[0] + '_f.jpg')

guilty_student_path = gb.glob(probe_path + student_pool_list[0] + '/*.jpg')
sample_1_student = gb.glob(probe_path + student_pool_list[1] + '/*.jpg')
sample_2_student = gb.glob(probe_path + student_pool_list[2] + '/*.jpg')
sample_3_student = gb.glob(probe_path + student_pool_list[3] + '/*.jpg')

print('For student ' + student_pool_list[0])
print('Similarity with his own probe photos: ')

guilty_student_emb = return_batch_embedding(senet50, guilty_student_path)
similarities = [cos_sim(real_embedding, guilty_student_emb[i]) for i in range(5)]
print(similarities)

print('Similarity with probe photos of ' + student_pool_list[1])

guilty_student_emb = return_batch_embedding(senet50, sample_1_student)
similarities = [cos_sim(real_embedding, guilty_student_emb[i]) for i in range(5)]
print(similarities)

print('Similarity with probe photos of ' + student_pool_list[2])

guilty_student_emb = return_batch_embedding(senet50, sample_2_student)
similarities = [cos_sim(real_embedding, guilty_student_emb[i]) for i in range(5)]
print(similarities)

print('Similarity with probe photos of ' + student_pool_list[3])

guilty_student_emb = return_batch_embedding(senet50, sample_3_student)
similarities = [cos_sim(real_embedding, guilty_student_emb[i]) for i in range(5)]
print(similarities)

For student 131
Similarity with his own probe photos: 
[0.843030289528448, 0.8113729719948266, 0.9253386365954008, 0.9951610047388882, 1.0158822016714377]
Similarity with probe photos of 3
[0.9164043200271385, 0.9639462859316921, 1.038463969553577, 0.7370212708938526, 0.9427439346104709]
Similarity with probe photos of 100
[0.9820348323326428, 0.8368485953419157, 0.7364532505336633, 0.8229633499185554, 0.7911821011541811]
Similarity with probe photos of 101
[0.915628988619557, 0.8509623225524872, 0.6868923944239351, 0.8403033583091135, 0.778768884138272]


In [26]:
# Experimentation
probe_path = img_path + 'probe/'
student_pool_list = gallery_names
accuracy = 0
error = []

for index, student in enumerate(student_pool_list):
    probe_list_students = student_pool_list.copy()
    watch_list_emb = []
    watch_list_images = []
    
    # First embeddings is always for the guilty student  
    guilty_student_image = probe_path + student + '/' + random.choice(os.listdir(probe_path + student))
    watch_list_emb.append(return_embedding(senet50,guilty_student_image))
    
    probe_list_students.remove(student)
    
    watch_list_students = random.sample(probe_list_students, 4)
    
    for suspect in watch_list_students:
        image = random.choice(os.listdir(probe_path + suspect))
        watch_list_images.append(probe_path + suspect + '/' + image)
        
    for suspect in watch_list_images:
        watch_list_emb.append(return_embedding(senet50,suspect))
    
    similarities = [cos_sim(face_embeddings[index], watch_list_emb[i]) for i in range(5)]
    
    # If the least distance is the first element, let's sum 1
    if np.argmin(np.array(similarities)) == 0:
        accuracy += 1
    else:
        error.append(student)
        #print('Error for student ' + student)
        #print(pd.DataFrame(similarities))
    #print('Student {}/{}'.format(index+1,len(student_pool_list)))
    #print('For student ' + student)
    #print(pd.DataFrame(similarities))
    #print('------------------------------------------------')
              
print("Accuracy: {0:.2f}".format(accuracy/len(student_pool_list)))
print('Error happend with the following students: ')
print(error)

Accuracy: 0.20
Error happend with the following students: 
['131', '3', '100', '101', '104', '108', '109', '10', '112', '113', '115', '117', '118', '11', '121', '123', '128', '12', '42', '45', '46', '49', '4', '55', '57', '59', '5', '6', '70', '75', '7', '80', '81', '8', '92', '93', '94', '96', '98', '9', '134', '135', '138', '139', '13', '145', '148', '149', '14', '151', '159', '15', '162', '165', '167', '16', '171', '172', '175', '177', '179', '181', '188', '18', '19', '24', '25', '27', '29', '2', '33', '34']
